In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import mysql.connector

In [2]:
sales_df = pd.read_csv('raw_data/Sales.csv')
stores_df = pd.read_csv('raw_data/Stores.csv')
products_df = pd.read_csv('raw_data/Products.csv')
customers_df = pd.read_csv('raw_data/Customers.csv', encoding ="ISO-8859-1")
exchange_df = pd.read_csv('raw_data/Exchange_Rates.csv')
print("All datasets loaded successfully!")

All datasets loaded successfully!


In [3]:
for df_name, df in {'Stores': stores_df, 'Products': products_df, 'Sales': sales_df, 
                    'Exchange': exchange_df, 'Customers': customers_df}.items():
    
    missing_count = df.isnull().sum()
    if missing_count.any():
        print(f"\nMissing values in {df_name} dataset:")
        print(missing_count[missing_count > 0])


Missing values in Stores dataset:
Square Meters    1
dtype: int64

Missing values in Sales dataset:
Delivery Date    49719
dtype: int64

Missing values in Customers dataset:
State Code    10
dtype: int64


In [4]:
stores_df['Open Date']

0      1/1/2008
1     1/12/2008
2      1/7/2012
3      1/1/2010
4     12/9/2015
        ...    
62     3/6/2008
63     1/1/2010
64     1/1/2012
65     1/1/2014
66     1/1/2010
Name: Open Date, Length: 67, dtype: object

In [5]:
stores_df['Open Date'] = pd.to_datetime(stores_df['Open Date'])

In [6]:
stores_df['Open Date']

0    2008-01-01
1    2008-01-12
2    2012-01-07
3    2010-01-01
4    2015-12-09
        ...    
62   2008-03-06
63   2010-01-01
64   2012-01-01
65   2014-01-01
66   2010-01-01
Name: Open Date, Length: 67, dtype: datetime64[ns]

In [7]:
stores_df['Square Meters']

0      595.0
1      665.0
2     2000.0
3     2000.0
4     2000.0
       ...  
62    2000.0
63    1330.0
64    1785.0
65     840.0
66       NaN
Name: Square Meters, Length: 67, dtype: float64

In [8]:
stores_df['Square Meters'].mean()

np.float64(1402.1969696969697)

In [9]:
stores_df['Square Meters'] = stores_df['Square Meters'].fillna(stores_df['Square Meters'].mean())

In [10]:
stores_df['Square Meters']

0      595.00000
1      665.00000
2     2000.00000
3     2000.00000
4     2000.00000
         ...    
62    2000.00000
63    1330.00000
64    1785.00000
65     840.00000
66    1402.19697
Name: Square Meters, Length: 67, dtype: float64

In [11]:
for df_name, df in {'Stores': stores_df, 'Products': products_df, 'Sales': sales_df, 
                    'Exchange': exchange_df, 'Customers': customers_df}.items():
    
    missing_count = df.isnull().sum()
    if missing_count.any():
        print(f"\nMissing values in {df_name} dataset:")
        print(missing_count[missing_count > 0])


Missing values in Sales dataset:
Delivery Date    49719
dtype: int64

Missing values in Customers dataset:
State Code    10
dtype: int64


In [12]:
pd.Timestamp.now() - stores_df['Open Date']

0    6227 days 15:21:55.517350
1    6216 days 15:21:55.517350
2    4760 days 15:21:55.517350
3    5496 days 15:21:55.517350
4    3328 days 15:21:55.517350
                ...           
62   6162 days 15:21:55.517350
63   5496 days 15:21:55.517350
64   4766 days 15:21:55.517350
65   4035 days 15:21:55.517350
66   5496 days 15:21:55.517350
Name: Open Date, Length: 67, dtype: timedelta64[ns]

In [13]:
stores_df['Store_Age_Years'] = (pd.Timestamp.now() - stores_df['Open Date']).dt.total_seconds() / (365.25 * 24 * 60 * 60)

In [14]:
stores_df['Store_Size_Category'] = pd.qcut(stores_df['Square Meters'], 
                                         q=4, 
                                         labels=['Small', 'Medium', 'Large', 'Extra Large'])

In [15]:
stores_df['Store_Age_Years']

0     17.050350
1     17.020233
2     13.033923
3     15.048981
4      9.113320
        ...    
62    16.872389
63    15.048981
64    13.050350
65    11.048981
66    15.048981
Name: Store_Age_Years, Length: 67, dtype: float64

In [16]:
for df_name, df in {'Stores': stores_df, 'Products': products_df, 'Sales': sales_df, 
                    'Exchange': exchange_df, 'Customers': customers_df}.items():
    
    missing_count = df.isnull().sum()
    if missing_count.any():
        print(f"\nMissing values in {df_name} dataset:")
        print(missing_count[missing_count > 0])


Missing values in Sales dataset:
Delivery Date    49719
dtype: int64

Missing values in Customers dataset:
State Code    10
dtype: int64


In [17]:
stores_df['Square Meters'].round(2)

0      595.0
1      665.0
2     2000.0
3     2000.0
4     2000.0
       ...  
62    2000.0
63    1330.0
64    1785.0
65     840.0
66    1402.2
Name: Square Meters, Length: 67, dtype: float64

In [18]:
products_df['Unit Price USD']
products_df['Unit Cost USD']

0        $6.62 
1        $6.62 
2        $7.40 
3       $11.00 
4       $11.00 
         ...   
2512    $43.07 
2513    $43.07 
2514     $1.71 
2515     $1.71 
2516     $1.71 
Name: Unit Cost USD, Length: 2517, dtype: object

In [19]:
for col in ['Unit Cost USD', 'Unit Price USD']:
    products_df[col] = products_df[col].str.replace('[\$,]', '', regex=True).astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\deepa\AppData\Local\Temp\ipykernel_17328\3904179585.py:2: SyntaxWarning: invalid escape sequence '\$'
  products_df[col] = products_df[col].str.replace('[\$,]', '', regex=True).astype(float)


In [20]:
products_df['Profit_Margin'] = ((products_df['Unit Price USD'] - products_df['Unit Cost USD']) / 
                               products_df['Unit Price USD'] * 100).round(2)

In [21]:
products_df['Profit_Margin']

0       49.04
1       49.04
2       49.04
3       49.00
4       49.00
        ...  
2512    66.87
2513    66.87
2514    48.96
2515    48.96
2516    48.96
Name: Profit_Margin, Length: 2517, dtype: float64

In [22]:
products_df['Price_Category'] = pd.qcut(products_df['Unit Price USD'], 
                                      q=5, 
                                      labels=['Budget', 'Economy', 'Standard', 'Premium', 'Luxury'])
products_df['Category_Subcategory'] = products_df['Category'] + ' - ' + products_df['Subcategory']

In [23]:
sales_df['Order Date'] = pd.to_datetime(sales_df['Order Date'])
sales_df['Year'] = sales_df['Order Date'].dt.year
sales_df['Month'] = sales_df['Order Date'].dt.month
sales_df['Month_Name'] = sales_df['Order Date'].dt.month_name()
sales_df['Quarter'] = sales_df['Order Date'].dt.quarter
sales_df['Day_of_Week'] = sales_df['Order Date'].dt.day_name()
sales_df['Order_Quarter'] = sales_df['Order Date'].dt.to_period('Q').astype(str)

In [24]:
if 'Delivery Date' in sales_df.columns:
    sales_df = sales_df.drop(columns=['Delivery Date'])

In [25]:
exchange_df['Date'] = pd.to_datetime(exchange_df['Date'])
exchange_df.rename(columns={'Currency': 'CurrencyCode'}, inplace=True)
exchange_df['Exchange'] = exchange_df['Exchange'].round(4)

In [26]:
customers_df['State Code'] = customers_df['State Code'].fillna('Unknown')
customers_df['Birthday'] = pd.to_datetime(customers_df['Birthday'])
customers_df['Age'] = (pd.Timestamp.now() - customers_df['Birthday']).dt.total_seconds() / (365.25 * 24 * 60 * 60)
customers_df['Age'] = customers_df['Age'].round().astype('Int64')
customers_df['Age_Category'] = pd.cut(customers_df['Age'], 
                                    bins=[0, 25, 35, 50, 65, 100],
                                    labels=['Young Adult', 'Adult', 'Middle Age', 'Senior', 'Elder'])
customers_df.rename(columns={'Zip Code ': 'Zip Code'}, inplace=True)
customers_df['Zip Code'] = pd.to_numeric(customers_df['Zip Code'], errors='coerce')
customers_df['Zip Code'] = customers_df['Zip Code'].astype('Int64')
customers_df['Customer_Region'] = customers_df['Country'] + ' - ' + customers_df['State']

In [27]:
for df_name, df in {'Stores': stores_df, 'Products': products_df, 'Sales': sales_df, 
                    'Exchange': exchange_df, 'Customers': customers_df}.items():
    
    missing_count = df.isnull().sum()
    if missing_count.any():
        print(f"\nMissing values in {df_name} dataset:")
        print(missing_count[missing_count > 0])


Missing values in Customers dataset:
Zip Code    4230
dtype: int64


In [28]:
sales_with_products = sales_df.merge(products_df[['ProductKey', 'Unit Price USD']], 
                                   on='ProductKey', how='left')
sales_df['Revenue_USD'] = sales_with_products['Unit Price USD'] * sales_df['Quantity']

In [29]:
sales_df['Revenue_USD']

0          68.00
1         854.00
2         665.94
3        2254.00
4         326.00
          ...   
62879     417.00
62880     480.00
62881     219.98
62882     140.26
62883    4753.00
Name: Revenue_USD, Length: 62884, dtype: float64

In [30]:
mysql_host = "localhost"
mysql_user = "root"
mysql_password = "simple"
mysql_database = "sales_record"
mysql_port = "3306"

In [33]:
import mysql.connector
try:
        connection = mysql.connector.connect(
            host=mysql_host,
            user=mysql_user,
            password=mysql_password,
            database=mysql_database,
            port=mysql_port
        )
        cursor = connection.cursor()
        print("Connected to MySQL database successfully")

        cursor.execute("CREATE DATABASE IF NOT EXISTS sales_record")
        cursor.execute("USE sales_record")

        print("\nCreating tables...")    
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Cleaned_Stores (
                StoreKey INT PRIMARY KEY,
                Country VARCHAR(255),
                State VARCHAR(255),
                SquareMeters DECIMAL(10, 2),
                OpenDate DATE,
                Store_Age_Years DECIMAL(10, 2),
                Store_Size_Category VARCHAR(50)
            )
        """)

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Cleaned_Products (
                ProductKey INT PRIMARY KEY,
                ProductName VARCHAR(255),
                Brand VARCHAR(255),
                Color VARCHAR(255),
                UnitCostUSD DECIMAL(10, 2),
                UnitPriceUSD DECIMAL(10, 2),
                SubcategoryKey INT,
                Subcategory VARCHAR(255),
                CategoryKey INT,
                Category VARCHAR(255),
                Profit_Margin DECIMAL(10, 2),
                Price_Category VARCHAR(50),
                Category_Subcategory VARCHAR(255)
            )
        """)

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Cleaned_Customers (
                CustomerKey INT PRIMARY KEY,
                Gender VARCHAR(10),
                Name VARCHAR(255),
                City VARCHAR(255),
                StateCode VARCHAR(50),
                State VARCHAR(255),
                ZipCode VARCHAR(20),
                Country VARCHAR(255),
                Continent VARCHAR(255),
                Birthday DATE,
                Age INT,
                Age_Category VARCHAR(50),
                Customer_Region VARCHAR(255)
            )
        """)

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Cleaned_Sales (
                OrderNumber INT,
                LineItem INT,
                OrderDate DATE,
                CustomerKey INT,
                StoreKey INT,
                ProductKey INT,
                Quantity INT,
                CurrencyCode VARCHAR(10),
                Year INT,
                Month INT,
                Month_Name VARCHAR(20),
                Quarter INT,
                Day_of_Week VARCHAR(20),
                Order_Quarter VARCHAR(10),
                Revenue_USD DECIMAL(10, 2),
                FOREIGN KEY (CustomerKey) REFERENCES Cleaned_Customers(CustomerKey),
                FOREIGN KEY (ProductKey) REFERENCES Cleaned_Products(ProductKey),
                FOREIGN KEY (StoreKey) REFERENCES Cleaned_Stores(StoreKey)
            )
        """)  

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Cleaned_Exchange_Rates (
                Date DATE,
                CurrencyCode VARCHAR(10),
                Exchange DECIMAL(10, 4)
            )
        """)
        print("Tables created successfully!")


        # Insert Stores data
        for _, row in stores_df.iterrows():
            values = []
            for value in row:
                if pd.isna(value):
                    values.append(None)
                else:
                    values.append(value)
                
            cursor.execute("""
                INSERT INTO Cleaned_Stores 
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, tuple(values))
    
        # Insert Products data
        for _, row in products_df.iterrows():
            values = []
            for value in row:
                if pd.isna(value):
                    values.append(None)
                else:
                    values.append(value)
            cursor.execute("""
                INSERT INTO Cleaned_Products 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, tuple(values))
    
        # Insert Customers data
        for _, row in customers_df.iterrows():
            values = []
            for value in row:
                if pd.isna(value):
                    values.append(None)
                else:
                    values.append(value)
            cursor.execute("""
                INSERT INTO Cleaned_Customers 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, tuple(values))
    
        # Insert Sales data
        for _, row in sales_df.iterrows():
            values = []
            for value in row:
                if pd.isna(value):
                    values.append(None)
                else:
                    values.append(value)
            cursor.execute("""
                INSERT INTO Cleaned_Sales 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, tuple(values))
    
        # Insert Exchange Rates data
        for _, row in exchange_df.iterrows():
            values = []
            for value in row:
                if pd.isna(value):
                    values.append(None)
                else:
                    values.append(value)
            cursor.execute("""
                INSERT INTO Cleaned_Exchange_Rates 
                VALUES (%s, %s, %s)
            """, tuple(values))
    
        # Commit all changes
        connection.commit()
        print("Data inserted successfully!")

except Exception as e:
    print(f"An error occurred: {str(e)}")
    if connection:
        connection.rollback()
finally:
    if connection:
        connection.close()    
        

Connected to MySQL database successfully

Creating tables...
Tables created successfully!
Data inserted successfully!


In [34]:
stores_df['Store_Size_Category']


0      Small
1      Small
2      Large
3      Large
4      Large
       ...  
62     Large
63    Medium
64     Large
65     Small
66     Large
Name: Store_Size_Category, Length: 67, dtype: category
Categories (4, object): ['Small' < 'Medium' < 'Large' < 'Extra Large']